# Default equation
def Eqn_Enz_MichaelisMenten_Activation_Allosteric(Conc_Enzyme, Conc_Substrate, Conc_Activator, kcat, KM, Ka, n):
    return (kcat * Conc_Enzyme * (1 + (Conc_Activator / Ka) ** n)) * (Conc_Substrate / (KM + Conc_Substrate))


In [46]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import Avogadro
from scipy import sqrt
import sympy as sm
# Micromolar conversion 
def uMtoCount(inVal:float):
    return inVal*10**-6*Avogadro
def counttouM(inVal:float):
    return inVal / Avogadro / 10**-6

In [13]:
# Where I got this code from:
# http://systems-sciences.uni-graz.at/etextbook/sw2/phpl_python.html

# define the system in this way (asuming a predator-prey-system with no negative values)
# to avoid interference x = r (for resource) and y = c (for consumer) 
r, c = sm.symbols('r, c', negative=False)
R = 2*r - r**2 - r*c
C = - c + r*c

# use sympy's way of setting equations to zero
REqual = sm.Eq(R, 0)
CEqual = sm.Eq(C, 0)

# compute fixed points
equilibria = sm.solve( (REqual, CEqual), r, c )
print(equilibria)

# needed to calculate (complex) squareroots of negative numbers
def eigenvalues(x,y):
    # 2*x - x**2 - x*y
    a11 = 2 - 2*x - y                # differentiated with respect to x
    a12 = - x                        # differentiated with respect to y
    # - y + x*y
    a21 = y                          # differentiated with respect to x
    a22 = - 1 + x                    # differentiated with respect to y

    tr = a11 + a22
    det = a11*a22 - a12*a21
    lambda1 = (tr - sqrt(tr**2 - 4*det))/2
    lambda2 = (tr + sqrt(tr**2 - 4*det))/2
    print('Check the fixed point  %s, %s' % (x,y)) 
    print('The real part of the first eigenvalue is %s' %lambda1.real)
    print('The real part of the second eigenvalue is %s' % lambda2.real)    
    
    if (lambda1.real < 0 and lambda2.real < 0):
        print('The fixed point in %s, %s is a sink. It is stable' % (x,y))
    if (lambda1.real > 0 and lambda2.real > 0):
        print('The fixed point in %s, %s is a source. It is unstable' % (x,y))
    if (lambda1.real > 0 and lambda2.real < 0):
        print('The fixed point in %s, %s is a saddle. It is unstable' % (x,y))
    if (lambda1.real < 0 and lambda2.real > 0):
        print('The fixed point in %s, %s is unstable' % (x,y))
    print('----------------------------')
    return lambda1 , lambda2

fp = []

# iterate through list of fixed points
for x,y in fp:
    eigenvalues(x,y)  


# here again, in order to avoid interference x = r (for resource) and y = c (for consumer) (see above) 

# put equations into matrix-form and compute Jacobian-matrix  
eqMat = sm.Matrix([ R, C ])
Mat = sm.Matrix([ r, c ])
jacMat = eqMat.jacobian(Mat)
print('Jacobian %s' % jacMat)
print('---------------------')

# iterate through list of equilibria
for item in equilibria:
    eqmat = jacMat.subs([ (r, item[0]), (c, item[1]) ])
    print('The eigenvalues for the fixed point (%s, %s) are %s:' 
          %(item[0], item[1], eqmat.eigenvals().keys()))
    print('-------------------------------------------')    

[(0, 0), (1, 1), (2, 0)]
Jacobian Matrix([[-c - 2*r + 2, -r], [c, r - 1]])
---------------------
The eigenvalues for the fixed point (0, 0) are dict_keys([2, -1]):
-------------------------------------------
The eigenvalues for the fixed point (1, 1) are dict_keys([-1/2 - sqrt(3)*I/2, -1/2 + sqrt(3)*I/2]):
-------------------------------------------
The eigenvalues for the fixed point (2, 0) are dict_keys([1, -2]):
-------------------------------------------


In [92]:
# constants

# kcats
KCAT_HEXOKINASE = 4e3
KCAT_PFK_ACTIVE = 1.7e4
KCAT_PFK_INACTIVE = 1.7e4
KCAT_PK = 170
# ks
K_SUMMARY_MAKE_PEPASE = 1e4

# KMs
KM_HEXOKINASE_GLUCOSE = uMtoCount(5)
KM_PFK_ACTIVE = uMtoCount(0.21)
KM_PFK_INACTIVE = uMtoCount(3.1e7)
KM_PK_PEP = uMtoCount(700)

# Concentrations
## Metabolites
CONC_GLUCOSE_T0 = uMtoCount(500)
CONC_ADP_T0 = uMtoCount(1700)
CONC_ATP_T0 = uMtoCount(5000)
## Intermediates
CONC_ATP = CONC_ATP_T0
CONC_ADP = CONC_ADP_T0
CONC_F6P = uMtoCount(15)
CONC_F16P = 0
CONC_PEP = 0
CONC_PYRUVATE = 0
## Enzymes
CONC_PFK = 4000 / Avogadro
CONC_PFK_ACTIVE = 0
CONC_PFK_INACTIVE = 0
CONC_HEXOKINASE = uMtoCount(0.01)
CONC_SUMMARY_MAKE_PEPASE = uMtoCount(1)
CONC_PYRUVATE_KINASE = uMtoCount(1)
CONC_PHOSPHOGLUCOSEISOMERASE = uMtoCount(1)

## Current exploratory values
KI_ATP_PYRUVATEKINASE = uMtoCount(5000)

In [4]:
# Attempting to base
def hexokinase(Conc_Substrate = CONC_GLUCOSE_T0, Conc_Enzyme = CONC_HEXOKINASE, kcat = KCAT_HEXOKINASE, KM = KM_HEXOKINASE_GLUCOSE):
    # Note: glucose is not decreasing...
    return (kcat * Conc_Enzyme * (Conc_Substrate / (KM + Conc_Substrate)))

def pfk_active(Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK_ACTIVE, kcat = KCAT_PFK_ACTIVE, KM = KM_PFK_ACTIVE):
    return (kcat * Conc_Enzyme * (Conc_Substrate / (KM + Conc_Substrate)))

def pfk_inactive(Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK_INACTIVE, kcat = KCAT_PFK_INACTIVE, KM = KM_PFK_INACTIVE):
    return (kcat * Conc_Enzyme * (Conc_Substrate / (KM + Conc_Substrate)))

def summaryMakePEPase(Conc_Substrate = CONC_F16P, k = K_SUMMARY_MAKE_PEPASE):
    return(Conc_Substrate * k)

def pyruvateKinase(Conc_Inhibitor = CONC_ATP, Ki = KI_ATP_PYRUVATEKINASE,Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK_ACTIVE, kcat = KCAT_PFK_ACTIVE, KM = KM_PFK_ACTIVE):
    return (kcat * (Conc_Enzyme / (1 + (Conc_Inhibitor / Ki)) * (Conc_Substrate / (KM + Conc_Substrate))))

In [5]:
# Attempting to regulate
def pfkActiveToInactiveRatio(ACTIVATING_COMPONENT, INHIBITING_COMPONENT, ACTIVATION_RATIO):
    return(ACTIVATION_RATIO * ACTIVATING_COMPONENT / INHIBITING_COMPONENT)    

In [32]:
CONC_F6P

9.033211139999999e+18

In [35]:

# (asuming no negative values)
# to avoid interference kcat = k and conc = c
a, i = sm.symbols('a, i', negative=False)
# PFK active
ACT = KCAT_PFK_ACTIVE * (4000 / (1 + i/a)) * CONC_F6P / (KM_PFK_ACTIVE + CONC_F6P) + KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)
# PFK inactive
INACT = KCAT_PFK_INACTIVE * (4000 / (1 + a/i)) * CONC_F6P / (KM_PFK_INACTIVE + CONC_F6P) + KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)

# use sympy's way of setting equations to zero
actEqual = sm.Eq(ACT, 0)
inactEqual = sm.Eq(INACT, 0)

# compute fixed points
equilibria = sm.solve( (actEqual, inactEqual), a, i )
print(equilibria)


# here again, in order to avoid interference x = r (for resource) and y = c (for consumer) (see above) 
# put equations into matrix-form and compute Jacobian-matrix  
eqMat = sm.Matrix([ ACT, INACT ])
Mat = sm.Matrix([ a, i ])
jacMat = eqMat.jacobian(Mat)
print('Jacobian %s' % jacMat)
print('---------------------')

# iterate through list of equilibria
for item in equilibria:
    eqmat = jacMat.subs([ (a, item[0]), (i, item[1]) ])
    print('The eigenvalues for the fixed point (%s, %s) are %s:' 
          %(item[0], item[1], eqmat.eigenvals().keys()))
    print('-------------------------------------------')  


[]
Jacobian Matrix([[67061143.9842209*i/(a**2*(1 + i/a)**2), -67061143.9842209/(a*(1 + i/a)**2)], [-32.9032098855436/(i*(a/i + 1)**2), 32.9032098855436*a/(i**2*(a/i + 1)**2)]])
---------------------


In [61]:
def searchConcentrationForEquilibriumPoints(substrateConcStart = uMtoCount(0.001), substrateConcEnd = uMtoCount(1000)):
    substrateConc = np.geomspace(substrateConcStart, substrateConcEnd, 7)    

    for conc in substrateConc:
        print(f'\nFor substrate concentration: {round(counttouM(conc), 4)} uM')
        # (asuming no negative values)
        # to avoid interference kcat = k and conc = c
        act, inact = sm.symbols('act, inact', negative=False)
        # PFK active
        ACT = -KCAT_PFK_ACTIVE * (4000 / ( 1 + inact / act)) * conc / (KM_PFK_ACTIVE + conc) + KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)
        INACT = -KCAT_PFK_INACTIVE * (4000 / ( 1 + act / inact)) * conc / (KM_PFK_INACTIVE + conc) + KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)

        # use sympy's way of setting equations to zero
        actEqual = sm.Eq(ACT, 0)
        inactEqual = sm.Eq(INACT, 0)

        # compute fixed points
        equilibria = sm.solve( (actEqual, inactEqual), act, inact )
        print('Fixed Points: ')
        print(equilibria)


        # here again, in order to avoid interference x = r (for resource) and y = c (for consumer) (see above) 
        # put equations into matrix-form and compute Jacobian-matrix  
        eqMat = sm.Matrix([ ACT, INACT ])
        Mat = sm.Matrix([ act, inact ])
        jacMat = eqMat.jacobian(Mat)
        print('Jacobian %s' % jacMat)
        print('---------------------')

        # iterate through list of equilibria
        for item in equilibria:
            eqmat = jacMat.subs([ (act, item[0]), (inact, item[1]) ])
            
            print('The eigenvalues for the fixed point (%s, %s) are %s:' 
                %(item[0], item[1], eqmat.eigenvals().keys()))
            print('-------------------------------------------')  



In [62]:
searchConcentrationForEquilibriumPoints()



For substrate concentration: 0.001 uM
Fixed Points: 
[]
Jacobian Matrix([[-322274.881516588*inact/(act**2*(1 + inact/act)**2), 322274.881516588/(act*(1 + inact/act)**2)], [0.00219354838702601/(inact*(act/inact + 1)**2), -0.00219354838702601*act/(inact**2*(act/inact + 1)**2)]])
---------------------

For substrate concentration: 0.01 uM
Fixed Points: 
[]
Jacobian Matrix([[-3090909.0909091*inact/(act**2*(1 + inact/act)**2), 3090909.0909091/(act*(1 + inact/act)**2)], [0.0219354838638918/(inact*(act/inact + 1)**2), -0.0219354838638918*act/(inact**2*(act/inact + 1)**2)]])
---------------------

For substrate concentration: 0.1 uM
Fixed Points: 
[]
Jacobian Matrix([[-21935483.8709677*inact/(act**2*(1 + inact/act)**2), 21935483.8709677/(act*(1 + inact/act)**2)], [0.219354838002081/(inact*(act/inact + 1)**2), -0.219354838002081*act/(inact**2*(act/inact + 1)**2)]])
---------------------

For substrate concentration: 1.0 uM
Fixed Points: 
[]
Jacobian Matrix([[-56198347.107438*inact/(act**2*(1 +

In [83]:

conc = uMtoCount(15.00)

print(f'\nFor substrate concentration: {round(counttouM(conc), 4)} uM')
# (asuming no negative values)
# to avoid interference kcat = k and conc = c
act, inact = sm.symbols('act, inact', negative=False)
# PFK active
ACT = KCAT_PFK_ACTIVE * (4000 / ( 1 + inact / act)) * conc / (KM_PFK_ACTIVE + conc) - KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)
INACT = KCAT_PFK_INACTIVE * (4000 / ( 1 + act / inact)) * conc / (KM_PFK_INACTIVE + conc) - KCAT_HEXOKINASE * CONC_HEXOKINASE / (CONC_GLUCOSE_T0 + KM_HEXOKINASE_GLUCOSE)

# use sympy's way of setting equations to zero
actEqual = sm.Eq(ACT, 0)
inactEqual = sm.Eq(INACT, 0)

# compute fixed points
equilibria = sm.solve( (actEqual, inactEqual), act, inact )
print('Fixed Points: ')
print(equilibria)


# here again, in order to avoid interference x = r (for resource) and y = c (for consumer) (see above) 
# put equations into matrix-form and compute Jacobian-matrix  
eqMat = sm.Matrix([ ACT, INACT ])
Mat = sm.Matrix([ act, inact ])
jacMat = eqMat.jacobian(Mat)
print('Jacobian %s' % jacMat)
print('---------------------')

# iterate through list of equilibria
for item in equilibria:
    eqmat = jacMat.subs([ (act, item[0]), (inact, item[1]) ])
    
    print('The eigenvalues for the fixed point (%s, %s) are %s:' 
        %(item[0], item[1], eqmat.eigenvals().keys()))
    print('-------------------------------------------')


For substrate concentration: 15.0 uM
Fixed Points: 
[]
Jacobian Matrix([[67061143.9842209*inact/(act**2*(1 + inact/act)**2), -67061143.9842209/(act*(1 + inact/act)**2)], [-32.9032098855436/(inact*(act/inact + 1)**2), 32.9032098855436*act/(inact**2*(act/inact + 1)**2)]])
---------------------


In [81]:
actEqual

Eq(0.0792079207920792 - 67061143.9842209/(1 + inact/act), 0)

In [82]:
inactEqual

Eq(0.0792079207920792 - 32.9032098855436/(act/inact + 1), 0)

In [75]:
sm.solve([actEqual], act, inact)


{act: 1.18112987908872e-9*inact}

In [76]:
sm.solve([inactEqual], act, inact)

{act: 414.403024804988*inact}

Do reduced model with only ATP/ADP sensitivity

In [ ]:
# Attempting to base -- just always assume the active site is saturated with P donor
def pfk_active(Conc_Substrate = CONC_F6P,Conc_Activator = 1, Ka = 1, Conc_Enzyme = CONC_PFK_ACTIVE, kcat = KCAT_PFK_ACTIVE, KM = KM_PFK_ACTIVE):
    return (kcat * Conc_Enzyme * ((1 + (Conc_Activator / Ka))) * (Conc_Substrate / (KM + Conc_Substrate)))

def pyruvateKinase(Conc_Inhibitor = CONC_ATP, Ki = KI_ATP_PYRUVATEKINASE,Conc_Substrate = CONC_F6P, Conc_Enzyme = CONC_PFK_ACTIVE, kcat = KCAT_PFK_ACTIVE, KM = KM_PFK_ACTIVE):
    return (kcat * (Conc_Enzyme / (1 + (Conc_Inhibitor / Ki)) * (Conc_Substrate / (KM + Conc_Substrate))))

In [ ]:
atp_Ki_PK = uMtoCount(5000) # made this one up
adp_ka_PFK = uMtoCount(74)  # This one is kinda pulled from some number in lit

conc = uMtoCount(15.00)

print(f'\nFor substrate concentration: {round(counttouM(conc), 4)} uM')
# (asuming no negative values)
# to avoid interference kcat = k and conc = c
pfk_conc, pk_conc = sm.symbols('pfk_conc, pk_conc', negative=False)
# PFK active
PFK = KCAT_PFK_ACTIVE * pfk_conc * (1 + (CONC_ADP/adp_ka_PFK)) * conc / (KM_PFK_ACTIVE + conc) 
PK = KCAT_PK * pk_conc / (1 + (CONC_ATP/KI_ATP_PYRUVATEKINASE)) * conc / (KM_PFK_INACTIVE + conc) 

# use sympy's way of setting equations to zero
actEqual = sm.Eq(ACT, 0)
inactEqual = sm.Eq(INACT, 0)

# compute fixed points
equilibria = sm.solve( (actEqual, inactEqual), act, inact )
print('Fixed Points: ')
print(equilibria)


# here again, in order to avoid interference x = r (for resource) and y = c (for consumer) (see above) 
# put equations into matrix-form and compute Jacobian-matrix  
eqMat = sm.Matrix([ ACT, INACT ])
Mat = sm.Matrix([ act, inact ])
jacMat = eqMat.jacobian(Mat)
print('Jacobian %s' % jacMat)
print('---------------------')

# iterate through list of equilibria
for item in equilibria:
    eqmat = jacMat.subs([ (act, item[0]), (inact, item[1]) ])
    
    print('The eigenvalues for the fixed point (%s, %s) are %s:' 
        %(item[0], item[1], eqmat.eigenvals().keys()))

In [114]:
#del equilibria
atp_Ki_PK = uMtoCount(50) # made this one up
adp_ka_PFK = uMtoCount(74)  # This one is kinda pulled from some number in lit

def searchConcentrationForEquilibriumPoints(substrateConcStart = 0.00001, substrateConcEnd = 10000000):
    substrateConc = np.geomspace(substrateConcStart, substrateConcEnd,13)    

    for conc in substrateConc:
        print(f'\nFor Vmax: {conc} uM')
        # (asuming no negative values)
        # to avoid interference kcat = k and conc = c
        pfk_conc, pk_conc = sm.symbols('pfk_conc, pk_conc', negative=False)
        # PFK active
        # For now just set the F6P = 15uM
        #PFK = KCAT_PFK_ACTIVE * pfk_conc * (1 + (CONC_ADP/adp_ka_PFK)) * uMtoCount(15) / (KM_PFK_ACTIVE + uMtoCount(15)) 
        #PK = KCAT_PK * pk_conc / (1 + (CONC_ATP/atp_Ki_PK)) * conc / (KM_PFK_INACTIVE + conc) 
        
        PFK =  pfk_conc * (1 + (CONC_ADP/adp_ka_PFK)) * uMtoCount(15) / (KM_PFK_ACTIVE + uMtoCount(15)) 
        PK =  pk_conc / (1 + (CONC_ATP/atp_Ki_PK)) * uMtoCount(5) / (KM_PFK_INACTIVE + uMtoCount(5)) 

        # use sympy's way of setting equations to zero
        pfkEqual = sm.Eq(PFK, 0)
        pkEqual = sm.Eq(PK, 0)

        # compute fixed points
        equilibria = sm.solve( (pfkEqual, pkEqual), pfk_conc, pk_conc )
        print('Fixed Points: ')
        print(equilibria)


        # here again, in order to avoid interference x = r (for resource) and y = c (for consumer) (see above) 
        # put equations into matrix-form and compute Jacobian-matrix  
        eqMat = sm.Matrix([ PFK, PK ])
        Mat = sm.Matrix([ pfk_conc, pk_conc ])
        jacMat = eqMat.jacobian(Mat)
        print('Jacobian %s' % jacMat)
        print('---------------------')

        # iterate through list of equilibria
        if equilibria is not None:
            for item in equilibria:
                eqmat = jacMat.subs([ (pfk_conc, item), (pk_conc, item) ])
                
                print('The eigenvalues for the fixed point (%s, %s) are %s:' 
                    %(item, item, eqmat.eigenvals().keys()))
                print('-------------------------------------------')  

searchConcentrationForEquilibriumPoints()


For Vmax: 1e-05 uM
Fixed Points: 
{pfk_conc: 0.0, pk_conc: 0.0}
Jacobian Matrix([[23.6419851804467, 0], [0, 1.59693362936714e-9]])
---------------------
The eigenvalues for the fixed point (pfk_conc, pfk_conc) are dict_keys([23.6419851804467, 1.59693362936714e-9]):
-------------------------------------------
The eigenvalues for the fixed point (pk_conc, pk_conc) are dict_keys([23.6419851804467, 1.59693362936714e-9]):
-------------------------------------------

For Vmax: 0.0001 uM
Fixed Points: 
{pfk_conc: 0.0, pk_conc: 0.0}
Jacobian Matrix([[23.6419851804467, 0], [0, 1.59693362936714e-9]])
---------------------
The eigenvalues for the fixed point (pfk_conc, pfk_conc) are dict_keys([23.6419851804467, 1.59693362936714e-9]):
-------------------------------------------
The eigenvalues for the fixed point (pk_conc, pk_conc) are dict_keys([23.6419851804467, 1.59693362936714e-9]):
-------------------------------------------

For Vmax: 0.001 uM
Fixed Points: 
{pfk_conc: 0.0, pk_conc: 0.0}
Ja

In [87]:
searchConcentrationForEquilibriumPoints()


For substrate concentration: 0.001 uM
Fixed Points: 
{pfk_conc: 0.0, pk_conc: 0.0}
Jacobian Matrix([[1931.47175611631, 0], [0, 2.74193548378252e-9]])
---------------------


TypeError: 'Symbol' object is not subscriptable

In [99]:
 # For now just set the F6P = 15uM
PFK = KCAT_PFK_ACTIVE * pfk_conc * (1 + (CONC_ADP/adp_ka_PFK)) * uMtoCount(15) / (KM_PFK_ACTIVE + conc) 
PK = KCAT_PK * pk_conc / (1 + (CONC_ATP/atp_Ki_PK)) * conc / (KM_PFK_INACTIVE + conc) 


# use sympy's way of setting equations to zero
pfkEqual = sm.Eq(PFK, 0)

pfkEqual

NameError: name 'pfk_conc' is not defined